Notebook to ascertain whether we can incorporate procedures into the context data for generating text.

In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2
import sqlalchemy
import string

In [2]:
dbschema='public'

#DEMO data
# cnx = sqlalchemy.create_engine('postgresql+psycopg2://btvdksxhyqtmjs:74db647f02a6f661c9a9ef888e24cebe9b17cfd2cc7566c51556096ec9977964\
# @ec2-54-211-255-161.compute-1.amazonaws.com/dc8973qq7atsb1', connect_args={'options': '-csearch_path={}'.format(dbschema)})

cnx = sqlalchemy.create_engine('postgresql+psycopg2://njolzisalnylqx:31b9211af0689fdc75c296546539fd3d86b8c8135d277924a0659bcfe735adb3\
@ec2-3-219-135-162.compute-1.amazonaws.com/d6di1ovvoadbog', connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [3]:
# procedures data

df_procedures = pd.read_sql_query('''
  SELECT p."SUBJECT_ID", p."HADM_ID", p."SEQ_NUM", p."ICD9_CODE", icd."SHORT_TITLE", icd."LONG_TITLE"
  FROM "PROCEDURES_ICD" p
  INNER JOIN "D_ICD_PROCEDURES" icd 
  USING ("ICD9_CODE")
  ORDER BY p."SUBJECT_ID", p."SEQ_NUM"
  --LIMIT 10000;
''', cnx)

print(df_procedures.shape)
df_procedures.head(20)

(246178, 6)


,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,2,163353,1,9955,Vaccination NEC,Prophylactic administration of vaccine against...
1,3,145834,1,9604,Insert endotracheal tube,Insertion of endotracheal tube
2,3,145834,2,9962,Heart countershock NEC,Other electric countershock of heart
3,3,145834,3,8964,Pulmon art wedge monitor,Pulmonary artery wedge monitoring
4,3,145834,4,9672,Cont inv mec ven 96+ hrs,Continuous invasive mechanical ventilation for...
5,3,145834,5,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified"
6,3,145834,6,966,Entral infus nutrit sub,Enteral infusion of concentrated nutritional s...
7,4,185777,1,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified"
8,4,185777,2,8872,Dx ultrasound-heart,Diagnostic ultrasound of heart
9,4,185777,3,3323,Other bronchoscopy,Other bronchoscopy


In [4]:
# procedures data

df_proc_sum = pd.read_sql_query('''
  SELECT "SUBJECT_ID", "HADM_ID", COUNT("SEQ_NUM") as "COUNT"
  FROM "PROCEDURES_ICD" p
  INNER JOIN "D_ICD_PROCEDURES" icd 
  USING ("ICD9_CODE")
  GROUP BY "SUBJECT_ID", "HADM_ID"
  ORDER BY COUNT("SEQ_NUM") DESC
  --LIMIT 10000;
''', cnx)

print(df_proc_sum.shape)
df_proc_sum.head(20)

(52234, 3)


,SUBJECT_ID,HADM_ID,COUNT
0,29467,194819,40
1,62795,173748,40
2,24810,143994,40
3,57764,103584,40
4,99544,145236,39
5,40548,168670,39
6,27755,155889,38
7,1427,178731,38
8,17029,190907,37
9,10187,138921,37


In [5]:
np.mean(df_proc_sum['COUNT'])

4.712983880231267

Each subject has on average 4.5 procedures during a hospital admission

In [6]:
df_adm = pd.read_sql_query('''
  SELECT n."SUBJECT_ID", n."HADM_ID", MAX(n."CHARTDATE") AS "CHARTDATE", MAX(a."DISCHTIME") AS "DISCHTIME",
  (cast(MAX(a."DISCHTIME") as date) - cast(MAX(n."CHARTDATE") as date)) AS "DIFF"
  FROM "NOTEEVENTS" n
  JOIN "ADMISSIONS" a
  ON n."SUBJECT_ID" = a."SUBJECT_ID" AND n."HADM_ID" = a."HADM_ID" 
  WHERE n."CATEGORY" = 'Discharge summary'
  GROUP BY n."SUBJECT_ID", n."HADM_ID"
  ORDER BY "DIFF" DESC
  --LIMIT 10000;
''', cnx)

print(df_adm.shape)
df_adm.head(20)

(52726, 5)


,SUBJECT_ID,HADM_ID,CHARTDATE,DISCHTIME,DIFF
0,6145,163196.0,2110-06-13,2110-09-02 15:35:00,81
1,15482,178068.0,2125-01-12,2125-02-20 13:45:00,39
2,22560,137561.0,2164-04-10,2164-05-18 19:00:00,38
3,6764,138049.0,2159-01-30,2159-03-08 14:00:00,37
4,632,104207.0,2159-11-01,2159-12-02 14:55:00,31
5,15006,155765.0,2189-12-23,2190-01-23 12:25:00,31
6,26446,150657.0,2152-04-07,2152-05-08 12:00:00,31
7,4604,113052.0,2116-04-15,2116-05-15 12:00:00,30
8,7648,118565.0,2130-09-27,2130-10-24 11:42:00,27
9,16727,157755.0,2158-09-01,2158-09-27 16:11:00,26


In [7]:
df_adm.loc[df_adm['DIFF'] < 0, 'DIFF'] = 0

In [8]:
np.mean(df_adm['DIFF'])

0.029435193263285666

Clearly there are a lot of patients who stay for a long time in hospital after being discharged from the ICU. However the average stay post ICU discharge is only 0.03 days. Meaning almost all patients leave hospital after being discharged from the ICU. What about if we discount patients who died in ICU.

In [9]:
df_death = pd.read_sql_query('''
  SELECT "SUBJECT_ID", "DOD"
  FROM "PATIENTS"
  ORDER BY "SUBJECT_ID"
  --LIMIT 10000;
''', cnx)

print(df_death.shape)
df_death.head()

(46520, 2)


,SUBJECT_ID,DOD
0,2,None
1,3,2102-06-14 00:00:00
2,4,None
3,5,None
4,6,None


In [10]:
df_temp = pd.merge(df_adm, df_death,  how='left', left_on=['SUBJECT_ID'], right_on = ['SUBJECT_ID'])
df_temp.head()

,SUBJECT_ID,HADM_ID,CHARTDATE,DISCHTIME,DIFF,DOD
0,6145,163196.0,2110-06-13,2110-09-02 15:35:00,81,2110-10-18 00:00:00
1,15482,178068.0,2125-01-12,2125-02-20 13:45:00,39,None
2,22560,137561.0,2164-04-10,2164-05-18 19:00:00,38,2164-05-18 00:00:00
3,6764,138049.0,2159-01-30,2159-03-08 14:00:00,37,None
4,632,104207.0,2159-11-01,2159-12-02 14:55:00,31,None


In [11]:
df_temp['DEAD'] = np.where(df_temp['DOD'] <= df_temp['CHARTDATE'], 1, 0)

In [12]:
df_temp2 = df_temp[df_temp['DEAD'] == 0]
df_temp2.shape

(52691, 7)

In [13]:
np.mean(df_temp2['DIFF'])

0.029454745592226377

It turns out this only barely makes a difference. Still, most patients are discharged immediately after their ICU stay. I have now realised that we are including neonates in this. Let's remove them to look at only adults.

In [14]:
sql = """
  SELECT DISTINCT p."SUBJECT_ID"
  FROM "PATIENTS" p 
  INNER JOIN "NOTEEVENTS" n 
  ON p."SUBJECT_ID" = n."SUBJECT_ID"
  WHERE ROUND((cast("CHARTDATE" as date) - cast("DOB" as date)) / 365.242,0) > 14
  AND n."CATEGORY" = 'Discharge summary'
  ORDER BY "SUBJECT_ID"
  --LIMIT 100;
"""

df = pd.read_sql_query(sqlalchemy.text(sql), cnx)
adults = list(df['SUBJECT_ID'])

In [15]:
df_temp3 = df_temp[df_temp['SUBJECT_ID'].isin(adults)]
print(df_temp3.shape)
df_temp3.head()

(48902, 7)


,SUBJECT_ID,HADM_ID,CHARTDATE,DISCHTIME,DIFF,DOD,DEAD
0,6145,163196.0,2110-06-13,2110-09-02 15:35:00,81,2110-10-18 00:00:00,0
2,22560,137561.0,2164-04-10,2164-05-18 19:00:00,38,2164-05-18 00:00:00,0
6,26446,150657.0,2152-04-07,2152-05-08 12:00:00,31,2152-05-08 00:00:00,0
8,7648,118565.0,2130-09-27,2130-10-24 11:42:00,27,2130-10-24 00:00:00,0
9,16727,157755.0,2158-09-01,2158-09-27 16:11:00,26,2165-02-19 00:00:00,0


In [16]:
np.mean(df_temp3['DIFF'])

0.017115864381824875

As predicted, the average stay post ICU discharge drops even further after removing neonates. Let's take a closer look at some of these patients who are having super long stays in hospital

In [17]:
df_procedures[df_procedures['HADM_ID'] == 150657]

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
137443,26446,150657,1,311,Temporary tracheostomy,Temporary tracheostomy
137444,26446,150657,2,3612,Aortocor bypas-2 cor art,(Aorto)coronary bypass of two coronary arteries
137445,26446,150657,3,341,Incision of mediastinum,Incision of mediastinum
137446,26446,150657,4,3723,Rt/left heart card cath,Combined right and left heart cardiac catheter...
137447,26446,150657,5,8856,Coronar arteriogr-2 cath,Coronary arteriography using two catheters
137448,26446,150657,6,8853,Lt heart angiocardiogram,Angiocardiography of left heart structures
137449,26446,150657,7,3761,Pulsation balloon implan,Implant of pulsation balloon
137450,26446,150657,9,4621,Temporary ileostomy,Temporary ileostomy
137451,26446,150657,10,4595,Anal anastomosis,Anastomosis to anus
137452,26446,150657,11,415,Total splenectomy,Total splenectomy


There aren't that many procedures, but still to be safe, we can only include procedures for patients who leave hopsital within a day of discharge.

In [18]:
df_final = df_temp3[df_temp3['DIFF'] <= 1]
print(df_final.shape)
df_final.head()

(48803, 7)


,SUBJECT_ID,HADM_ID,CHARTDATE,DISCHTIME,DIFF,DOD,DEAD
213,43430,131076.0,2182-07-08,2182-07-09 10:46:00,1,None,0
214,16412,171732.0,2195-09-07,2195-09-08 12:30:00,1,2198-08-26 00:00:00,0
215,4826,101492.0,2174-06-11,2174-06-12 14:09:00,1,2174-08-02 00:00:00,0
216,22115,102011.0,2191-03-24,2191-03-25 14:11:00,1,None,0
217,3911,155474.0,2141-03-29,2141-03-30 12:03:00,1,2141-04-17 00:00:00,0


In [19]:
df_final.HADM_ID.unique().shape[0]

48803

The total number of unique hospital admission ids in our discharge summary dataset is 48,902. So we are only losing out on 99 discharge summaries for context data - not the end of the world by any means. Let's export the hospital admission ids to use in our preprocessing. However, we'll still only use it only for the last discharge note of an admission - just to be safe

In [20]:
df_final[['HADM_ID']].to_csv('../data/df_proc_hadm_ids.csv',index=False)